In [2]:
%matplotlib inline

In [3]:
from __future__ import print_function, division, unicode_literals
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import seaborn as sns
import torch
import os


In [4]:
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
device

device(type='cuda')

In [5]:
picture_frames = pd.read_csv('train.csv')
picture_name = picture_frames.iloc[:,0]
label = picture_frames.iloc[:,1]
picture_name = np.asarray(picture_name)
label = np.asarray(label)
picture_name

array(['1.png', '2.png', '3.png', ..., '7558.png', '7559.png', '7560.png'],
      dtype=object)

In [15]:
class CellPictureDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, root_dir, csv_file=None, transform=None, test=False):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        if csv_file is not None:
            self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.test = test

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.data_frame.iloc[idx, 0])
        image = io.imread(img_name)
        if self.transform:
            self.image = self.transform(image)
        if self.test is False:
            label = self.data_frame.iloc[idx, 1]
            # label = np.asarray(label)
            # self.label = torch.from_numpy(label).unsqueeze_(0)
            sample = [self.image, self.label]
            return sample
        else:
            return self.image

In [16]:
data_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])])

In [17]:
train_dataset = CellPictureDataset('train/', csv_file='train.csv', transform=data_transform)
test_dataset = CellPictureDataset('train/',transform=data_transform, test=True)
for i in range(2):
    data = train_dataset[i]
    print(i, data)

0 [tensor([[[ 0.6941,  0.5529,  0.4275,  ...,  0.8353,  0.8745,  0.8824],
         [ 0.5843,  0.5216,  0.4902,  ...,  0.9059,  0.9294,  0.9373],
         [ 0.4667,  0.4902,  0.5451,  ...,  0.9765,  0.9843,  0.9843],
         ...,
         [ 0.8745,  0.8745,  0.8745,  ...,  0.3333,  0.2314,  0.1686],
         [ 0.8667,  0.8745,  0.8745,  ...,  0.3725,  0.3804,  0.3098],
         [ 0.8588,  0.8588,  0.8667,  ...,  0.3255,  0.3333,  0.2863]],

        [[ 0.4353,  0.2706,  0.1451,  ...,  0.7725,  0.8118,  0.8196],
         [ 0.3176,  0.2392,  0.2000,  ...,  0.8431,  0.8745,  0.8824],
         [ 0.1922,  0.2000,  0.2392,  ...,  0.9137,  0.9294,  0.9294],
         ...,
         [ 0.8667,  0.8588,  0.8588,  ...,  0.0275, -0.0745, -0.1373],
         [ 0.8588,  0.8588,  0.8667,  ...,  0.0588,  0.0667, -0.0039],
         [ 0.8431,  0.8510,  0.8588,  ...,  0.0039,  0.0196, -0.0275]],

        [[ 0.8588,  0.7176,  0.6000,  ...,  0.8431,  0.8745,  0.8745],
         [ 0.7804,  0.7255,  0.6863,  ...,

In [ ]:
import collections
class MyConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(collections.OrderedDict([
            ('conv1', nn.Conv2d(3,64,3,padding=1)),
            ('relu1', nn.ReLU()),
            ('conv2', nn.Conv2d(64,64,3,padding=1)),
            ('relu2', nn.ReLU()),
            ('conv2', nn.Conv2d(64,32,3,padding=1)),
            ('relu2', nn.ReLU()),
            # Put in a linear layers ...
            ('flatten', nn.Flatten()),                                          
            ('fc1', nn.Linear(32*10000,1028)),
            ('relu1', nn.ReLU()),
            ('fc2', nn.Linear(1028,4)),
        ]))

    def forward(self, x):
        x = self.model(x)
        return x



In [ ]:
model = MyConvNet()
data = torch.ones(10,3,100,100)
result = model(data)
result.shape

torch.Size([10, 4])

In [ ]:
train_loader = DataLoader(cell_dataset, batch_size=4, shuffle=True, num_workers=0)